This is the jupyterr notebook for our results section


In [29]:
#Requirements
import numpy as np
import plotly.graph_objects as go
from scipy.optimize import minimize
import pandas as pd
import statsmodels.api as sm
from scipy.stats import mannwhitneyu, levene, f_oneway
import prospect_optimizer as po
import os
import sys
notebook_dir = os.getcwd()
parent_dir = os.path.join(notebook_dir, "..")

In [30]:

# Example usage
r_s = np.array([[0.1, 0.03, 0.01], [0.07, 0.03, 0.04], [-0.03, 0.03, 0.03]])  # Returns for each asset across time periods
r_hat = 0.0  # Reference return
lambda_ = 2  # Base loss aversion coefficient
strategy = "conservative"  # You can switch between "aggressive" or "conservative"

result = po.optimize_portfolio(r_s, r_hat, lambda_, strategy)
optimized_weights = [round(weight, 4) for weight in result.x]
print("Optimized weights:", optimized_weights)

Optimized weights: [0.4977, 0.5023, -0.0]


In [31]:
#Indlæser returnsdata
datapath = os.path.join(parent_dir+'/data/returns_data.csv')


returns = pd.read_csv(datapath, index_col='Date')

In [32]:
returns

,SPY,sp600,MSCI_World,10Y_Bond_Return,Gold
Date,,,,,
1989-01-31,NaN,NaN,NaN,0.007508,-0.039
1989-02-28,-0.028944,0.003491,-0.007820,0.007767,-0.016
1989-03-31,0.020806,0.015736,-0.008079,0.007750,-0.009
1989-04-30,0.050090,0.047130,0.021454,0.007517,-0.012
1989-05-31,0.035137,0.035508,-0.026060,0.007167,-0.042
...,...,...,...,...,...
2024-07-31,0.000000,0.000000,0.000000,0.003408,0.041
2024-08-31,0.000000,0.000000,0.000000,0.003258,0.037
2024-09-30,0.000000,0.000000,0.000000,0.003175,0.044


In [33]:
"""
Her er der eksempel på hvordan koden kan bruges

"""
lambda_values = [1.5,1.75,1.99,2.25,2.5]
gamma_values = [0.12,0.2,0.25,0.35,0.5]

results = po.resultgenerator(lambda_values, gamma_values, returns)
aggressive_results =results[0]
conservative_results = results[1]

/Users/oliver/Desktop/GitHub/speciale_repo/functions/prospect_optimizer.py:219: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebalancing_dates = returns.loc[returns.index[0] + lookback_offset:].resample(rebalancing_freq).first().index
/Users/oliver/Desktop/GitHub/speciale_repo/functions/prospect_optimizer.py:219: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebalancing_dates = returns.loc[returns.index[0] + lookback_offset:].resample(rebalancing_freq).first().index
/Users/oliver/Desktop/GitHub/speciale_repo/functions/prospect_optimizer.py:219: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  rebalancing_dates = returns.loc[returns.index[0] + lookback_offset:].resample(rebalancing_freq).first().index
/Users/oliver/Desktop/GitHub/speciale_repo/functions/prospect_optimizer.py:219: FutureWarning: 'M' is deprecated and will be re

##Now we conduct statistical tests for the portfolios

In [34]:
# Extract and display all keys in aggressive_results

aggressive_results =results[0]
conservative_results = results[1]


# Extract and display all keys in aggressive_results
aggressive_keys = [list(result.keys())[0] for result in aggressive_results]
print("Aggressive Results Keys:")
for i, key in enumerate(aggressive_keys, start=1):
    print(f"{i}: {key}")

# Extract and display all keys in conservative_results
conservative_keys = [list(result.keys())[0] for result in conservative_results]
print("Conservative Results Keys:")
for i, key in enumerate(conservative_keys, start=1):
    print(f"{i}: {key}")

specific_portfolios = [
    (("aggressive_lambda1.5_gamma0.12", aggressive_results[0]["aggressive_result_lambda1.5_gamma0.12"]),
     ("conservative_lambda1.5_gamma0.5", conservative_results[4]["conservative_result_lambda1.5_gamma0.5"])),
    (("aggressive_lambda2.25_gamma0.12", aggressive_results[15]["aggressive_result_lambda2.25_gamma0.12"]),
     ("conservative_lambda2.25_gamma0.12", conservative_results[15]["conservative_result_lambda2.25_gamma0.12"])),
    (("aggressive_lambda1.99_gamma0.12", aggressive_results[10]["aggressive_result_lambda1.99_gamma0.12"]),
     ("conservative_lambda1.99_gamma0.12", conservative_results[10]["conservative_result_lambda1.99_gamma0.12"]))
]

# Perform the Mann-Whitney U test on specific portfolios
mann_whitney_results = po.perform_mann_whitney(specific_portfolios)

# Perform the volatility test on specific portfolios
volatility_results = po.test_volatility(specific_portfolios)

# Display the results
print("Mann-Whitney Results:")
print(mann_whitney_results)
print("\nVolatility Test Results:")
print(volatility_results)

# Save the results to CSV files for further analysis
#mann_whitney_results.to_csv('mann_whitney_results_specific.csv', index=False)
#volatility_results.to_csv('volatility_results_specific.csv', index=False)

Aggressive Results Keys:
1: aggressive_result_lambda1.5_gamma0.12
2: aggressive_result_lambda1.5_gamma0.2
3: aggressive_result_lambda1.5_gamma0.25
4: aggressive_result_lambda1.5_gamma0.35
5: aggressive_result_lambda1.5_gamma0.5
6: aggressive_result_lambda1.75_gamma0.12
7: aggressive_result_lambda1.75_gamma0.2
8: aggressive_result_lambda1.75_gamma0.25
9: aggressive_result_lambda1.75_gamma0.35
10: aggressive_result_lambda1.75_gamma0.5
11: aggressive_result_lambda1.99_gamma0.12
12: aggressive_result_lambda1.99_gamma0.2
13: aggressive_result_lambda1.99_gamma0.25
14: aggressive_result_lambda1.99_gamma0.35
15: aggressive_result_lambda1.99_gamma0.5
16: aggressive_result_lambda2.25_gamma0.12
17: aggressive_result_lambda2.25_gamma0.2
18: aggressive_result_lambda2.25_gamma0.25
19: aggressive_result_lambda2.25_gamma0.35
20: aggressive_result_lambda2.25_gamma0.5
21: aggressive_result_lambda2.5_gamma0.12
22: aggressive_result_lambda2.5_gamma0.2
23: aggressive_result_lambda2.5_gamma0.25
24: aggressi

In [35]:
aggressive_results

[{'aggressive_result_lambda1.5_gamma0.12':             Portfolio Returns  Compounded Returns  \
  Date                                                
  1991-01-31           0.015278            1.015278   
  1991-02-28           0.067281            1.083587   
  1991-03-31           0.022203            1.107646   
  1991-04-30           0.000320            1.108000   
  1991-05-31           0.006717            1.115442   
  ...                       ...                 ...   
  2023-08-31          -0.016350           26.237150   
  2023-09-30          -0.001862           26.188299   
  2023-10-31           0.026161           26.873413   
  2023-11-30           0.009917           27.139919   
  2023-12-31           0.022000           27.736997   
  
                               Portfolio Weights     r_hat       SPY     sp600  \
  Date                                                                           
  1991-01-31           [1.2, 0.2, 0.2, 1.2, 1.2]  0.024828  0.041518  0.07068